In [1]:
%matplotlib inline

from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, Merge, Reshape 
from keras.layers.core import Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D, Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras import regularizers
from keras.callbacks import ModelCheckpoint
from keras.layers.merge import concatenate

from sklearn.preprocessing import binarize
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score

import numpy as np
import scipy as sp
import scipy.io as sio

import h5py

import seaborn as sns
import matplotlib.pyplot as plt

import pickle

from deepsea import *

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 980 Ti (CNMeM is disabled, cuDNN 5005)


# Data loading

In [2]:
test_X, dmy = loaddata("data/deepsea/", "test")
train_X, dmy = loaddata("data/deepsea/", "train")
valid_X, dmy = loaddata("data/deepsea/", "valid")

In [3]:
test_marks = np.load("data/deepersea/testx.npy")
train_marks = np.load("data/deepersea/trainx.npy")
valid_marks = np.load("data/deepersea/validx.npy")

In [4]:
test_Y = np.load("data/deepersea/testy.npy") 
train_Y = np.load("data/deepersea/trainy.npy") 
valid_Y = np.load("data/deepersea/validy.npy") 

# Constructing a data generator

Keras model can be trained both on a numpy array and a keras Iterator.  
Here we implement our own custom iterator.

In [5]:
from keras.preprocessing.image import Iterator

In [6]:
class DataIterator(Iterator):
    
    def __init__(self, seq, mark, y, batch_size=32, shuffle=False, seed=None):
        self.seq = seq
        self.mark = mark
        self.y = y
        super(DataIterator, self).__init__(mark.shape[0], batch_size, shuffle, seed)
        
    def next(self):
        # Keeps under lock only the mechanism which advances
        # the indexing of each batch.
        with self.lock:
            index_array, current_index, current_batch_size = next(self.index_generator)
        
        seqindex =  [i%self.seq.shape[0] for i in index_array]
        
        seq_batch = self.seq[seqindex, :]
        mark_batch = self.mark[index_array, :]
        
        if self.y is None:
            return [seq_batch, mark_batch]
        batch_y = self.y[index_array]
        return  [seq_batch, mark_batch], batch_y

In [31]:
valid = DataIterator(valid_X, valid_marks, valid_Y, batch_size=256)
train = DataIterator(train_X, train_marks, train_Y, batch_size=256)

# Defining a network model

In [32]:
def DeeperSEA():
    nkernels = [320,480,960]
    nhidden = [15, 15]
    in_size1 = (1,1000,4)
    in_size2 = (11, )
    l2_lam = 5e-07 
    l1_lam = 1e-08 
    
    #Generating convolution arm 
    model1 = Sequential()
    model1.add(Conv2D(nkernels[0], kernel_size=(1,8), strides=(1,1), padding='same', input_shape=in_size1, kernel_regularizer=regularizers.l2(l2_lam)))
    model1.add(Activation('relu'))
    model1.add(MaxPooling2D(pool_size=(1,4), strides=(1,4)))
    model1.add(Dropout(0.2))
    
    model1.add(Conv2D(nkernels[1], kernel_size=(1,8), strides=(1,1), padding='same', kernel_regularizer=regularizers.l2(l2_lam)))
    model1.add(Activation('relu'))
    model1.add(MaxPooling2D(pool_size=(1,4), strides=(1,4)))
    model1.add(Dropout(0.2))

    model1.add(Conv2D(nkernels[1], kernel_size=(1,8), strides=(1,1), padding='same', kernel_regularizer=regularizers.l2(l2_lam)))
    model1.add(Activation('relu'))
    model1.add(Dropout(0.5))
    model1.add(Flatten())
    print model1.output_shape 

    model2 = Sequential()
    model2.add(Dense(nhidden[0], kernel_regularizer=regularizers.l1(l1_lam), input_shape=in_size2))
    model2.add(Activation('relu'))
    model2.add(Dense(nhidden[1], kernel_regularizer=regularizers.l1(l1_lam)))
    model2.add(Activation('relu'))
    print model2.output_shape
    
    model = Sequential()
    
    #TODO fix deprecation
    model.add(Merge([model1, model2], mode="concat"))
    print model.output_shape
    
    model.add(Dense(919, kernel_regularizer=regularizers.l1(l1_lam)))
    model.add(Activation('relu'))
    model.add(Dense(1, kernel_regularizer=regularizers.l1(l1_lam)))
    model.add(Activation('sigmoid'))
    
    return model

In [33]:
model = DeeperSEA()

(None, 29760)
(None, 15)
(None, 29775)


/home/hiranumn/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:37: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [34]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [35]:
filepath="deepsea-{epoch:02d}-{val_loss:.2f}-full.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
result = model.fit_generator(train, train_marks.shape[0]/256, epochs=10, verbose=0,
                             validation_data=valid, validation_steps=valid_marks.shape[0]/256,
                             callbacks=callbacks_list)